In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import os
import scipy.stats as stats

In [14]:
# Define data directory
base_dir = "../results/experiment-001"
# Find all folders (models + control)
model_folders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]


## Trapezoid Rule

We analyze each csv for each model. We use the Trapezoid rule to calculate the Total Energy (Joules) for a specific trial and we repeat 30 Times: Do this for all 30 CSVs in that model's folder. 

In [ ]:
# We will store every single trial's final result in this list
all_trials_data = []

print("Processing all CSV files using the Trapezoid Rule...")

for folder in model_folders:
    # Find all 30 CSVs in this folder
    csv_pattern = os.path.join(base_dir, folder, "trial-*.csv")
    csv_files = glob.glob(csv_pattern)
    
    for file in csv_files:
        df = pd.read_csv(file)
        
        # Failsafe: check if the GPU power column exists
        if 'GPU0_POWER (mWatts)' not in df.columns:
            continue
            
        # Convert timestamp to elapsed Seconds (starting from 0)
        time_sec = (df['Time'] - df['Time'].min()) / 1000.0
        
        # Convert milliWatts to standard Watts
        power_watts = df['GPU0_POWER (mWatts)'] / 1000.0
        
        # Calculate Area Under the Curve (Total Joules)
        total_energy_joules = np.trapezoid(y=power_watts, x=time_sec)
        
        # Get the total duration of this specific trial
        total_time_seconds = time_sec.max()
        
        # Store the calculated metrics for THIS trial
        all_trials_data.append({
            'Model': folder,
            'Trial_Name': os.path.basename(file),
            'Total_Energy_Joules': total_energy_joules,
            'Total_Time_Seconds': total_time_seconds
        })

# Convert the master list into a DataFrame
df_results = pd.DataFrame(all_trials_data)

print(f"\nSuccessfully processed {len(df_results)} trials.")
print("\nHere is a sneak peek of the dataset:")
display(df_results.head())

## Anomalies Detection

Running the following block of code, we identified that 5 out of our 210 executions were anomalous. Four trials reached our 60-second hardware timeout limit and were forcefully aborted, resulting in extreme energy spikes. One trial probably experienced a software crash (terminating in 0.6 seconds). Following standard data-cleaning procedures, these outliers were discarded to prevent skewed distributions. Furthermore, sensor data confirmed that the GPU temperature never exceeded 66°C, validating that our cooling and resting strategy effectively prevented thermal throttling.

In [ ]:
print("=== OUTLIER DETECTION REPORT ===\n")
valid_trials = []

# Flag and filter outliers per model
for model in df_results['Model'].unique():
    model_df = df_results[df_results['Model'] == model].copy()
    
    # Calculate Mean and Standard Deviation for Time
    mean_time = model_df['Total_Time_Seconds'].mean()
    std_time = model_df['Total_Time_Seconds'].std()
    
    # We use different rules for the control group vs the AI models
    if model == 'control':
        # For the control group, we filter out extreme crashes and anomalies > 3 standard deviations
        outlier_condition = (
            (model_df['Total_Time_Seconds'] < 1.0) | 
            (np.abs(model_df['Total_Time_Seconds'] - mean_time) > 3 * std_time)
        )
    else:
        # For the AI models, we filter out crashes, timeouts (>= 59s), and anomalies > 3 standard deviations
        outlier_condition = (
            (model_df['Total_Time_Seconds'] < 1.0) | 
            (model_df['Total_Time_Seconds'] >= 59.0) |
            (np.abs(model_df['Total_Time_Seconds'] - mean_time) > 3 * std_time)
        )
    
    outliers = model_df[outlier_condition]
    clean_data = model_df[~outlier_condition]
    valid_trials.append(clean_data)
    
    if not outliers.empty and model != 'control':
        print(f"⚠️ {model} had {len(outliers)} anomalous runs discarded:")
        for _, row in outliers.iterrows():
            print(f"   - {row['Trial_Name']}: Time = {row['Total_Time_Seconds']:.2f}s, Energy = {row['Total_Energy_Joules']:.0f} J")
        print()

# Create your final, clean dataset!
clean_df = pd.concat(valid_trials)

print(f"✅ Data cleaning complete. Retained {len(clean_df)} valid trials out of {len(df_results)} total.")

### Checking Temperature

In [ ]:
print("Scanning temperature sensors across all trials...")

max_temps = {}
global_max = 0

# Loop through every single CSV and extract the maximum temperature
for folder in model_folders:
    csv_files = glob.glob(os.path.join(base_dir, folder, "trial-*.csv"))
    folder_max = 0
    
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            if 'GPU0_TEMPERATURE' in df.columns:
                trial_max = df['GPU0_TEMPERATURE'].max()
                
                # Update the max for this specific model
                if trial_max > folder_max:
                    folder_max = trial_max
                
                # Update the absolute global max
                if trial_max > global_max:
                    global_max = trial_max
        except Exception:
            continue
            
    max_temps[folder] = folder_max

# Print the final conclusive statement
print(f"\nThe absolute Maximum GPU Temperature recorded was {global_max}°C.")
if global_max <= 66:
    print("Conclusion Validated: Thermal throttling was successfully prevented.")

# Visual proof for the report
plt.figure(figsize=(12, 6))
bars = plt.bar(max_temps.keys(), max_temps.values(), color='#e74c3c', edgecolor='black', alpha=0.8)

plt.axhline(y=85, color='red', linestyle='--', linewidth=2, label='Typical Thermal Throttling Limit (85°C)')

plt.title("Peak GPU Temperature per AI Model During Execution", fontsize=15, fontweight='bold')
plt.ylabel("Maximum Temperature (°C)", fontsize=12)
plt.xticks(rotation=35, ha='right', fontsize=10)

plt.ylim(0, 100) 
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(loc='upper left')

# Add the exact temperature number on top of each bar
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 1.5, f"{int(yval)}°C", ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

## Exploratory Data Analysis

In [ ]:
plot_df = clean_df[clean_df['Model'] != 'control'].copy()

# Shorten the names to make the X-axis readable
plot_df['Model_Short'] = plot_df['Model'].str.replace('deepseek-r1_8b-llama-distill-', 'DS-')
plot_df['Model_Short'] = plot_df['Model_Short'].str.replace('llama3.1_8b-instruct-', 'Llama-')

# We group 4-bit together, then 8-bit, then 16-bit
quantization_order = [
    'Llama-q4_K_M', 'DS-q4_K_M', 
    'Llama-q8_0',   'DS-q8_0', 
    'Llama-fp16',   'DS-fp16'
]

plt.figure(figsize=(14, 8))
sns.set_theme(style="whitegrid")

# Create the Violin Plot, passing the new order
sns.violinplot(
    x='Model_Short', 
    y='Total_Energy_Joules', 
    data=plot_df, 
    order=quantization_order,
    inner=None, 
    color="lightgray", 
    linewidth=1.5,
    alpha=0.6
)

# Overlay the Box Plot, passing the SAME order so they align perfectly
sns.boxplot(
    x='Model_Short', 
    y='Total_Energy_Joules', 
    data=plot_df, 
    order=quantization_order, 
    width=0.2, 
    boxprops={'zorder': 2, 'alpha': 0.9},
    palette="Set2" 
)

plt.title("Energy Consumption Distribution (Grouped by Quantization)", fontsize=16, fontweight='bold')
plt.xlabel("Model Version (4-bit, 8-bit, 16-bit)", fontsize=12)
plt.ylabel("Total Energy (Joules)", fontsize=12)

plt.xticks(rotation=20, ha='right', fontsize=11)

plt.tight_layout()
plt.show()

## Shapiro - Wilk test & T-test for Normal or Mann-Whitney U Test for not Normal

In [ ]:
# Define the pairs we want to compare in a list
model_pairs = [
    ('llama3.1_8b-instruct-q4_K_M', 'deepseek-r1_8b-llama-distill-q4_K_M', '4-bit'),
    ('llama3.1_8b-instruct-q8_0', 'deepseek-r1_8b-llama-distill-q8_0', '8-bit'),
    ('llama3.1_8b-instruct-fp16', 'deepseek-r1_8b-llama-distill-fp16', '16-bit')
]

for llama_name, ds_name, precision in model_pairs:
    print(f"======================================================")
    print(f"=== STATISTICAL SIGNIFICANCE TEST: {precision.upper()} MODELS ===")
    print(f"======================================================\n")
    
    # Isolate the data for the current pair
    llama_data = clean_df[clean_df['Model'] == llama_name]['Total_Energy_Joules']
    ds_data = clean_df[clean_df['Model'] == ds_name]['Total_Energy_Joules']
    
    # 1. The Normality Test (Shapiro-Wilk)
    stat_l, p_l = stats.shapiro(llama_data)
    stat_d, p_d = stats.shapiro(ds_data)
    
    print(f"Shapiro-Wilk p-value (Llama): {p_l:.5f}")
    print(f"Shapiro-Wilk p-value (DeepSeek): {p_d:.5f}")
    
    # 2. Choose the right test dynamically based on the Professor's rules
    if p_l > 0.05 and p_d > 0.05:
        print("-> Both p-values are > 0.05. The data IS normally distributed.")
        print("-> Action: Using the Parametric Welch's T-Test.\n")
        stat_val, p_value = stats.ttest_ind(llama_data, ds_data, equal_var=False)
        test_name = "Welch's T-Test"
        
        # Use Means for the Effect Size since data is normal
        val_llama = llama_data.mean()
        val_ds = ds_data.mean()
        metric_name = "Mean"
    else:
        print("-> At least one p-value is < 0.05. The data is NOT normally distributed.")
        print("-> Action: Using the Non-Parametric Mann-Whitney U Test.\n")
        stat_val, p_value = stats.mannwhitneyu(llama_data, ds_data, alternative='two-sided')
        test_name = "Mann-Whitney U Test"
        
        # Use Medians for the Effect Size since data is heavily skewed
        val_llama = llama_data.median()
        val_ds = ds_data.median()
        metric_name = "Median"
        
    # 3. Print Final Significance
    print(f"{test_name} p-value: {p_value:.5e}")
    if p_value < 0.05:
        print(f"-> CONCLUSION: The energy difference between the {precision} models is STATISTICALLY SIGNIFICANT.\n")
    else:
        print(f"-> CONCLUSION: The difference is not statistically significant (just random noise).\n")

    # 4. Effect Size (Percentage Change)
    percent_diff = ((val_llama - val_ds) / val_llama) * 100

    print("=== EFFECT SIZE (PERCENTAGE CHANGE) ===")
    print(f"Llama {metric_name} Energy: {val_llama:.1f} Joules")
    print(f"DeepSeek {metric_name} Energy: {val_ds:.1f} Joules")
    
    if percent_diff > 0:
        print(f"✅ RESULT: DeepSeek is {percent_diff:.1f}% MORE ENERGY EFFICIENT than Llama.\n\n")
    else:
        print(f"✅ RESULT: Llama is {abs(percent_diff):.1f}% MORE ENERGY EFFICIENT than DeepSeek.\n\n")

#### WITHIN-FAMILY QUANTIZATION EFFECTIVENESS ANALYSIS


In [ ]:
# ============================================================================
# WITHIN-FAMILY QUANTIZATION EFFECTIVENESS ANALYSIS
# ============================================================================

print("\n" + "="*70)
print("QUANTIZATION EFFECTIVENESS: DOES SMALLER = LESS ENERGY?")
print("="*70 + "\n")

# Define families
families = {
    'Llama': ['llama3.1_8b-instruct-fp16', 'llama3.1_8b-instruct-q8_0', 'llama3.1_8b-instruct-q4_K_M'],
    'DeepSeek': ['deepseek-r1_8b-llama-distill-fp16', 'deepseek-r1_8b-llama-distill-q8_0', 'deepseek-r1_8b-llama-distill-q4_K_M']
}

quantization_results = []

for family_name, models in families.items():
    print(f"\n{'='*70}")
    print(f"   {family_name.upper()} FAMILY")
    print(f"{'='*70}\n")
    
    # Extract data
    fp16_data = clean_df[clean_df['Model'] == models[0]]['Total_Energy_Joules']
    q8_data = clean_df[clean_df['Model'] == models[1]]['Total_Energy_Joules']
    q4_data = clean_df[clean_df['Model'] == models[2]]['Total_Energy_Joules']
    
    # Test 1: fp16 vs q8
    print(f"[Test 1] {family_name} fp16 vs q8")
    _, p_fp16 = stats.shapiro(fp16_data)
    _, p_q8 = stats.shapiro(q8_data)
    
    if p_fp16 > 0.05 and p_q8 > 0.05:
        stat, p_val = stats.ttest_ind(fp16_data, q8_data, equal_var=False)
        test_name = "Welch's t-test"
        val_fp16, val_q8 = fp16_data.mean(), q8_data.mean()
        metric = "Mean"
    else:
        stat, p_val = stats.mannwhitneyu(fp16_data, q8_data, alternative='two-sided')
        test_name = "Mann-Whitney U"
        val_fp16, val_q8 = fp16_data.median(), q8_data.median()
        metric = "Median"
    
    reduction_q8 = ((val_fp16 - val_q8) / val_fp16) * 100
    print(f"  {metric}: fp16={val_fp16:.1f}J, q8={val_q8:.1f}J")
    print(f"  Reduction: {reduction_q8:.1f}% | p={p_val:.5f} | {'✅ Significant' if p_val < 0.05 else '❌ Not significant'}\n")
    
    # Test 2: fp16 vs q4
    print(f"[Test 2] {family_name} fp16 vs q4")
    _, p_q4 = stats.shapiro(q4_data)
    
    if p_fp16 > 0.05 and p_q4 > 0.05:
        stat, p_val = stats.ttest_ind(fp16_data, q4_data, equal_var=False)
        test_name = "Welch's t-test"
        val_fp16, val_q4 = fp16_data.mean(), q4_data.mean()
        metric = "Mean"
    else:
        stat, p_val = stats.mannwhitneyu(fp16_data, q4_data, alternative='two-sided')
        test_name = "Mann-Whitney U"
        val_fp16, val_q4 = fp16_data.median(), q4_data.median()
        metric = "Median"
    
    reduction_q4 = ((val_fp16 - val_q4) / val_fp16) * 100
    print(f"  {metric}: fp16={val_fp16:.1f}J, q4={val_q4:.1f}J")
    print(f"  Reduction: {reduction_q4:.1f}% | p={p_val:.5f} | {'✅ Significant' if p_val < 0.05 else '❌ Not significant'}\n")
    
    # Test 3: q8 vs q4
    print(f"[Test 3] {family_name} q8 vs q4")
    
    if p_q8 > 0.05 and p_q4 > 0.05:
        stat, p_val = stats.ttest_ind(q8_data, q4_data, equal_var=False)
        test_name = "Welch's t-test"
        val_q8, val_q4 = q8_data.mean(), q4_data.mean()
        metric = "Mean"
    else:
        stat, p_val = stats.mannwhitneyu(q8_data, q4_data, alternative='two-sided')
        test_name = "Mann-Whitney U"
        val_q8, val_q4 = q8_data.median(), q4_data.median()
        metric = "Median"
    
    reduction_q8_to_q4 = ((val_q8 - val_q4) / val_q8) * 100
    print(f"  {metric}: q8={val_q8:.1f}J, q4={val_q4:.1f}J")
    print(f"  Reduction: {reduction_q8_to_q4:.1f}% | p={p_val:.5f} | {'✅ Significant' if p_val < 0.05 else '❌ Not significant'}\n")
    
    # Store for summary
    quantization_results.append({
        'Family': family_name,
        'fp16→q8': reduction_q8,
        'fp16→q4': reduction_q4,
        'q8→q4': reduction_q8_to_q4
    })

# Summary table
print("\n" + "="*70)
print("SUMMARY: Energy Reduction from Quantization (%)")
print("="*70)
df_quant_summary = pd.DataFrame(quantization_results)
print(df_quant_summary.to_string(index=False))

#### HEATMAP: Energy Savings vs fp16 Baseline


In [ ]:
# ============================================================================
# HEATMAP: Energy Savings vs fp16 Baseline
# ============================================================================

plt.figure(figsize=(10, 5))

# Prepare data for heatmap
heatmap_data = []
for family_name, models in families.items():
    fp16_median = clean_df[clean_df['Model'] == models[0]]['Total_Energy_Joules'].median()
    q8_median = clean_df[clean_df['Model'] == models[1]]['Total_Energy_Joules'].median()
    q4_median = clean_df[clean_df['Model'] == models[2]]['Total_Energy_Joules'].median()
    
    savings_q8 = ((fp16_median - q8_median) / fp16_median) * 100
    savings_q4 = ((fp16_median - q4_median) / fp16_median) * 100
    
    heatmap_data.append([savings_q8, savings_q4])

df_heatmap = pd.DataFrame(heatmap_data, 
                          columns=['8-bit (q8)', '4-bit (q4)'],
                          index=['Llama 3.1', 'DeepSeek-R1'])

# Plot
sns.heatmap(df_heatmap, annot=True, fmt='.1f', cmap='RdYlGn', 
            center=0, vmin=-5, vmax=75,
            cbar_kws={'label': 'Energy Reduction (%)'}, 
            linewidths=2, linecolor='white')

plt.title('Energy Savings vs FP16 Baseline', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Model Family', fontsize=12, fontweight='bold')
plt.xlabel('Quantization Level', fontsize=12, fontweight='bold')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n✅ Interpretation:")
print("   - Green = Good (energy saved)")
print("   - Red = Bad (energy increased)")
print("   - Values show % reduction compared to fp16 baseline")

#### AVERAGE POWER (W) ANALYSIS


In [ ]:
# ============================================================================
# AVERAGE POWER (W) ANALYSIS
# ============================================================================

# Calculate average power
clean_df['Avg_Power_W'] = clean_df['Total_Energy_Joules'] / clean_df['Total_Time_Seconds']

print("\n" + "="*70)
print("AVERAGE POWER CONSUMPTION (W)")
print("="*70 + "\n")

power_summary = clean_df.groupby('Model')['Avg_Power_W'].agg(['mean', 'median', 'std'])
power_summary = power_summary.round(2)
power_summary = power_summary.sort_values('median')

print(power_summary)

# Plot
plt.figure(figsize=(12, 6))
plot_df = clean_df.copy()
plot_df['Model_Short'] = plot_df['Model'].str.replace('deepseek-r1_8b-llama-distill-', 'DS-')
plot_df['Model_Short'] = plot_df['Model_Short'].str.replace('llama3.1_8b-instruct-', 'Llama-')

quantization_order = [
    'Llama-q4_K_M', 'DS-q4_K_M', 
    'Llama-q8_0',   'DS-q8_0', 
    'Llama-fp16',   'DS-fp16'
]

sns.boxplot(x='Model_Short', y='Avg_Power_W', data=plot_df, 
            order=quantization_order, palette='Set2')

plt.title('Average Power Consumption by Model', fontsize=16, fontweight='bold')
plt.xlabel('Model Version', fontsize=12)
plt.ylabel('Average Power (Watts)', fontsize=12)
plt.xticks(rotation=20, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ Lower power = gentler on hardware, better for sustained workloads")

#### ENERGY VS TIME TRADE-OFF


In [ ]:
# ============================================================================
# ENERGY VS TIME TRADE-OFF
# ============================================================================

plt.figure(figsize=(12, 7))

# Color mapping
color_map = {
    'llama3.1_8b-instruct-q4_K_M': '#2ca02c',
    'deepseek-r1_8b-llama-distill-q4_K_M': '#98df8a',
    'llama3.1_8b-instruct-q8_0': '#ff7f0e',
    'deepseek-r1_8b-llama-distill-q8_0': '#ffbb78',
    'llama3.1_8b-instruct-fp16': '#d62728',
    'deepseek-r1_8b-llama-distill-fp16': '#ff9896'
}

label_map = {
    'llama3.1_8b-instruct-q4_K_M': 'Llama q4',
    'deepseek-r1_8b-llama-distill-q4_K_M': 'DeepSeek q4',
    'llama3.1_8b-instruct-q8_0': 'Llama q8',
    'deepseek-r1_8b-llama-distill-q8_0': 'DeepSeek q8',
    'llama3.1_8b-instruct-fp16': 'Llama fp16',
    'deepseek-r1_8b-llama-distill-fp16': 'DeepSeek fp16'
}

for model in clean_df['Model'].unique():
    if model == 'control':
        continue
    data = clean_df[clean_df['Model'] == model]
    plt.scatter(data['Total_Time_Seconds'], data['Total_Energy_Joules'],
               label=label_map[model], color=color_map[model], 
               s=80, alpha=0.6, edgecolors='black', linewidth=0.5)

plt.xlabel('Execution Time (seconds)', fontsize=13, fontweight='bold')
plt.ylabel('Total Energy (Joules)', fontsize=13, fontweight='bold')
plt.title('Energy vs Time Trade-off\n(Lower-left = Best: Fast + Efficient)', 
          fontsize=16, fontweight='bold', pad=20)
plt.legend(loc='upper left', framealpha=0.9, fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ Interpretation:")
print("   - Lower-left corner = BEST (fast + energy-efficient)")
print("   - Upper-right corner = WORST (slow + energy-hungry)")
print("   - Check if quantization makes models faster or slower")

#### ENERGY DELAY PRODUCT (EDP) - Overall Efficiency Metric


In [ ]:
# ============================================================================
# ENERGY DELAY PRODUCT (EDP) - Overall Efficiency Metric
# ============================================================================

# Calculate EDP (penalizes slow execution)
clean_df['EDP'] = clean_df['Total_Energy_Joules'] * (clean_df['Total_Time_Seconds'] ** 2)

print("\n" + "="*70)
print("ENERGY DELAY PRODUCT (EDP) - Lower = Better")
print("="*70 + "\n")

edp_summary = clean_df.groupby('Model')['EDP'].agg(['mean', 'median', 'std'])
edp_summary = edp_summary.sort_values('median')
edp_summary = edp_summary.round(1)

print(edp_summary)

# Plot
plt.figure(figsize=(12, 6))
plot_df = clean_df.copy()
plot_df['Model_Short'] = plot_df['Model'].str.replace('deepseek-r1_8b-llama-distill-', 'DS-')
plot_df['Model_Short'] = plot_df['Model_Short'].str.replace('llama3.1_8b-instruct-', 'Llama-')

sns.boxplot(x='Model_Short', y='EDP', data=plot_df, 
            order=quantization_order, palette='coolwarm')

plt.title('Energy Delay Product (EDP)\nLower = Better Overall Efficiency', 
          fontsize=16, fontweight='bold')
plt.xlabel('Model Version', fontsize=12)
plt.ylabel('EDP (J·s²)', fontsize=12)
plt.xticks(rotation=20, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ EDP = Energy × Time²")
print("   Penalizes models that are slow, even if energy-efficient")